In [1]:
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll


pd.set_option('display.max_columns', None)

Tentando encontrar o id do LeBron James atraves do módulo players da api

In [2]:
jogadores_encontrados = players.find_players_by_full_name('LeBron James')

if jogadores_encontrados:
    lebron = jogadores_encontrados[0]
    lebron_id = lebron['id']
    print(f"Encontramos LeBron James! ID: {lebron_id}")
    print(lebron) # Mostra todas as informações que a API tem sobre ele
else:
    print("Jogador LeBron James não encontrado.")

Encontramos LeBron James! ID: 2544
{'id': 2544, 'full_name': 'LeBron James', 'first_name': 'LeBron', 'last_name': 'James', 'is_active': True}


Buscando as estátisticas da ultima temporada para LeBron James

In [7]:

if 'lebron_id' in locals():
    print(f"Buscando logs de jogos para LeBron James (ID: {lebron_id}) na temporada 2024-25...")

    gamelog_lebron = playergamelog.PlayerGameLog(player_id=lebron_id, season='2024-25')

    df_lebron_2024 = gamelog_lebron.get_data_frames()[0]
    print("Dados da temporada 2024-25 carregados com sucesso!")

    display(df_lebron_2024.head())
    df_lebron_2024.info()


Buscando logs de jogos para LeBron James (ID: 2544) na temporada 2024-25...
Dados da temporada 2024-25 carregados com sucesso!


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22024,2544,0022401185,"Apr 11, 2025",LAL vs. HOU,W,22,6,11,0.545,1,4,0.250,1,1,1.000,0,4,4,8,1,0,1,1,14,5,1
1,22024,2544,0022401159,"Apr 09, 2025",LAL @ DAL,W,36,11,20,0.550,0,4,0.000,5,6,0.833,2,5,7,3,1,0,2,2,27,18,1
2,22024,2544,0022401153,"Apr 08, 2025",LAL @ OKC,L,35,8,19,0.421,3,6,0.500,9,10,0.900,0,7,7,3,1,0,6,1,28,-23,1
3,22024,2544,0022401135,"Apr 06, 2025",LAL @ OKC,W,34,9,16,0.563,1,1,1.000,0,0,0.000,2,1,3,7,1,0,2,0,19,7,1
4,22024,2544,0022401126,"Apr 04, 2025",LAL vs. NOP,W,33,9,17,0.529,2,7,0.286,7,8,0.875,0,0,0,8,2,0,1,1,27,7,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEASON_ID        70 non-null     object 
 1   Player_ID        70 non-null     int64  
 2   Game_ID          70 non-null     object 
 3   GAME_DATE        70 non-null     object 
 4   MATCHUP          70 non-null     object 
 5   WL               70 non-null     object 
 6   MIN              70 non-null     int64  
 7   FGM              70 non-null     int64  
 8   FGA              70 non-null     int64  
 9   FG_PCT           70 non-null     float64
 10  FG3M             70 non-null     int64  
 11  FG3A             70 non-null     int64  
 12  FG3_PCT          70 non-null     float64
 13  FTM              70 non-null     int64  
 14  FTA              70 non-null     int64  
 15  FT_PCT           70 non-null     float64
 16  OREB             70 non-null     int64  
 17  DREB             7

Validado que a Api está funcionando! 

Nesse momento, vou escolher 3 jogadores mais famosos (all-stars) e definir um range maior de temporadas para fazer alguns testes

vou pegar 3 temporadas completas de 2022-22, 2023-24, 2024-25

In [12]:
players.find_players_by_full_name('Stephen Curry')

[{'id': 201939,
  'full_name': 'Stephen Curry',
  'first_name': 'Stephen',
  'last_name': 'Curry',
  'is_active': True}]

In [13]:
players.find_players_by_full_name('Kevin Durant')

[{'id': 201142,
  'full_name': 'Kevin Durant',
  'first_name': 'Kevin',
  'last_name': 'Durant',
  'is_active': True}]

LeBron, Stephen e Kevin

In [14]:
ids_jogadores = [2544, 201939, 201142]

# Lista de temporadas que quero buscar
temporadas = ['2022-23', '2023-24', '2024-25']

In [15]:
import time # Importa a biblioteca para fazer pausas pois as vezes a API pode limitar devido ao excesso de requisições

todos_os_logs = []

# Loop passando pelas temporadas
for temporada in temporadas:
    # Loop pelos IDs
    for jogador_id in ids_jogadores:
        print(f"jogador {jogador_id} na temporada {temporada}...")
        try:
            gamelog = playergamelog.PlayerGameLog(player_id=jogador_id, season=temporada)
            df_gamelog = gamelog.get_data_frames()[0]
            
            todos_os_logs.append(df_gamelog)
            
            print(f" -> Dados encontrados: {len(df_gamelog)} jogos.")
            
        except Exception as e:
            print(f" -> Erro ao buscar dados para jogador {jogador_id} na temporada {temporada}: {e}")
            
        #Espera 0.6 segundos entre cada requisição para não sobrecarregar a API
        time.sleep(0.6) 

if todos_os_logs:
    # Junta todos os DataFrames da lista em um único DataFrame gigante
    df_completo_raw = pd.concat(todos_os_logs, ignore_index=True)
    print("\nDados de todos os jogadores e temporadas foram combinados!")
    display(df_completo_raw.head())
    df_completo_raw.info()
else:
    print("\nNenhum dado foi coletado. :(")

jogador 2544 na temporada 2022-23...
 -> Dados encontrados: 55 jogos.
jogador 201939 na temporada 2022-23...
 -> Dados encontrados: 56 jogos.
jogador 201142 na temporada 2022-23...
 -> Dados encontrados: 47 jogos.
jogador 2544 na temporada 2023-24...
 -> Dados encontrados: 71 jogos.
jogador 201939 na temporada 2023-24...
 -> Dados encontrados: 74 jogos.
jogador 201142 na temporada 2023-24...
 -> Dados encontrados: 75 jogos.
jogador 2544 na temporada 2024-25...
 -> Dados encontrados: 70 jogos.
jogador 201939 na temporada 2024-25...
 -> Dados encontrados: 70 jogos.
jogador 201142 na temporada 2024-25...
 -> Dados encontrados: 62 jogos.

Dados de todos os jogadores e temporadas foram combinados!


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,2544,0022201228,"Apr 09, 2023",LAL vs. UTA,W,33,13,25,0.520,8,14,0.571,2,2,1.0,1,5,6,6,1,1,2,0,36,20,1
1,22022,2544,0022201212,"Apr 07, 2023",LAL vs. PHX,W,29,6,19,0.316,3,7,0.429,1,2,0.5,0,6,6,6,0,0,5,1,16,11,1
2,22022,2544,0022201196,"Apr 05, 2023",LAL @ LAC,L,35,13,20,0.650,4,6,0.667,3,5,0.6,0,8,8,7,1,1,6,0,33,-10,1
3,22022,2544,0022201186,"Apr 04, 2023",LAL @ UTA,W,38,14,27,0.519,3,10,0.300,6,6,1.0,0,5,5,6,1,1,5,2,37,-7,1
4,22022,2544,0022201171,"Apr 02, 2023",LAL @ HOU,W,29,8,18,0.444,1,7,0.143,1,1,1.0,2,8,10,11,0,1,1,1,18,23,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEASON_ID        580 non-null    object 
 1   Player_ID        580 non-null    int64  
 2   Game_ID          580 non-null    object 
 3   GAME_DATE        580 non-null    object 
 4   MATCHUP          580 non-null    object 
 5   WL               580 non-null    object 
 6   MIN              580 non-null    int64  
 7   FGM              580 non-null    int64  
 8   FGA              580 non-null    int64  
 9   FG_PCT           580 non-null    float64
 10  FG3M             580 non-null    int64  
 11  FG3A             580 non-null    int64  
 12  FG3_PCT          580 non-null    float64
 13  FTM              580 non-null    int64  
 14  FTA              580 non-null    int64  
 15  FT_PCT           580 non-null    float64
 16  OREB             580 non-null    int64  
 17  DREB            

Salvando todos os dados agora em data/raw para evitar a perda e garantir reprodutibilidade antes de limpezas e modificações


In [16]:
import os

pasta_dados_brutos = '../data/raw'
os.makedirs(pasta_dados_brutos, exist_ok=True) 

caminho_arquivo_raw = os.path.join(pasta_dados_brutos, 'nba_player_gamelogs_raw.csv')

# Salva o DataFrame como um arquivo CSV
# index=False evita que o índice do Pandas seja salvo como uma coluna no CSV
if 'df_completo_raw' in locals() and not df_completo_raw.empty:
    df_completo_raw.to_csv(caminho_arquivo_raw, index=False)
    print(f"\nDados brutos salvos com sucesso em: {caminho_arquivo_raw}")
else:
    print("\nDataFrame raw está vazio ou não existe. Nenhum arquivo foi salvo.")


Dados brutos salvos com sucesso em: ../data/raw/nba_player_gamelogs_raw.csv


O grande problema de utilizar dados de apenas 3 jogadores é que o modelo que será criado no futuro nao será tão bom generalizando.
Ele saberá prever padrões muito bem definidos para os 3 jogadores, mas para outros em posições e estilos diferentes teremos problemas.
Isso vai dar muito viès de amostra. Precisamos de pelo menos 100 jogadores diferentes em posições diferentes (pivô, alas, armadores) com funções distintas (estrelas, titulares, jogadores do banco, defensores, arremessadores) e times diferentes.

então, ja na pasta /src vou selecionar 100 jogadores distintos da nba_api e criar um novo resultado em data/raw.